In [67]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize our sentence
sentences = [
    "I love my dog",
    "I love my cat",
    "You love my dog!",
    "YOu really goof me off?!"
]

test_data = ["i really love my dog huh?",
    "my dog loves my manatee"
]

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'love': 2,
 'my': 3,
 'i': 4,
 'dog': 5,
 'you': 6,
 'cat': 7,
 'really': 8,
 'goof': 9,
 'me': 10,
 'off': 11}

In [68]:
# Manage sequencing, creating sequences of words in each sentence
sequences = tokenizer.texts_to_sequences(sentences)
sequences

[[4, 2, 3, 5], [4, 2, 3, 7], [6, 2, 3, 5], [6, 8, 9, 10, 11]]

In [69]:
# missing data in training set, use `oov_token="<OOV>"`
test_seq = tokenizer.texts_to_sequences(test_data)
test_seq

[[4, 8, 2, 3, 5, 1], [3, 5, 1, 3, 1]]

In [70]:
# Add padding, make every sentence same length by padding zeros
padded = pad_sequences(sequences, padding="post", truncating="post", maxlen=6)
padded

array([[ 4,  2,  3,  5,  0,  0],
       [ 4,  2,  3,  7,  0,  0],
       [ 6,  2,  3,  5,  0,  0],
       [ 6,  8,  9, 10, 11,  0]])

In [71]:
# get our dataset
import json

datastore = []
with open("20_Sarcasm_Headlines_Dataset_v2.txt", "r") as f:
    for x in f.readlines():
        datastore.append(eval(x))

sentences, labels, urls = [], [], []

for item in datastore:
    sentences.append(item["headline"])
    labels.append(item["is_sarcastic"])
    urls.append(item["article_link"])

sentences[:5]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep. totally nails why congress is falling short on gender, racial equality',
 'eat your veggies: 9 deliciously different recipes',
 'inclement weather prevents liar from getting to work',
 "mother comes pretty close to using word 'streaming' correctly"]

In [72]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [73]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [75]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [76]:
training_sequences[:2], training_padded[0]

([[1, 325, 3169, 5817, 2489, 3, 655, 993],
  [5818, 1723, 735, 2490, 47, 248, 11, 1824, 919, 8, 1825, 2032, 2297]],
 array([   1,  325, 3169, 5817, 2489,    3,  655,  993,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]))

In [74]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [77]:
# Embedding and model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [78]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [80]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Epoch 1/30
625/625 [==============================] - 1s 2ms/step - loss: 0.1518 - accuracy: 0.9459 - val_loss: 0.3743 - val_accuracy: 0.8445
Epoch 2/30
625/625 [==============================] - 1s 2ms/step - loss: 0.1373 - accuracy: 0.9510 - val_loss: 0.3820 - val_accuracy: 0.8460
Epoch 3/30
625/625 [==============================] - 1s 2ms/step - loss: 0.1271 - accuracy: 0.9556 - val_loss: 0.4020 - val_accuracy: 0.8402
Epoch 4/30
625/625 [==============================] - 1s 2ms/step - loss: 0.1161 - accuracy: 0.9607 - val_loss: 0.4228 - val_accuracy: 0.8372
Epoch 5/30
625/625 [==============================] - 1s 2ms/step - loss: 0.1075 - accuracy: 0.9635 - val_loss: 0.4359 - val_accuracy: 0.8390
Epoch 6/30
625/625 [==============================] - 1s 2ms/step - loss: 0.0999 - accuracy: 0.9671 - val_loss: 0.4809 - val_accuracy: 0.8284
Epoch 7/30
625/625 [==============================] - 1s 2ms/step - loss: 0.0914 - accuracy: 0.9707 - val_loss: 0.4807 - val_accuracy: 0.8334
Epoch 

In [105]:
sacarstic_sentence = [
    "I dear you not you lil goof, shame on you",
    "Ok I like it",
    "Do you hate me? Yes",
    "I Loved it!!!",
    "Jason",
    "Paul is good",
    "Fuck"
]

sequences = tokenizer.texts_to_sequences(sacarstic_sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(np.round(model.predict(padded), 3))

[[0.001]
 [0.   ]
 [0.   ]
 [0.006]
 [0.002]
 [0.2  ]
 [1.   ]]
